In [ ]:
import numpy as np
import random
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

import seaborn as sns
from data_preprocessing import load_combined_data
import torch.nn.functional as F

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False  # disables optimizations for reproducibility
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(42)

In [ ]:
# Function to load preprocessed data
combined_data, combined_labels = load_combined_data(data_path='ES_Down_combined_data.csv', labels_path='ES_Down_combined_labels.csv',downsample = True)

In [ ]:

# Map labels from original multi-class to binary
def map_labels(original_labels,classification_type='binary'):
    if classification_type == 'binary':
        label_map = {2: 0, 3: 1}  # 0: tightening, 1: untightening
        num_classes = 2
    return np.array([label_map[label] if label in label_map else -1 for label in original_labels]),num_classes

# Filter and preprocess the data
binary_labels,num_of_classes = map_labels(combined_labels,classification_type='binary')
valid_indices = binary_labels != -1
X = combined_data[valid_indices, :, 1:]  # exclude timestamps
y = binary_labels[valid_indices]

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y) # can be removed as our code uses binary labels directly

# Convert to tensors
X_tensor = torch.FloatTensor(X)
y_tensor = torch.LongTensor(y_encoded)

# ---- 1. Split into train/test ----
X_train, X_test, y_train, y_test = train_test_split(
    X_tensor, y_tensor, test_size=0.2, stratify=y_tensor)


# ---- 2. Split train into labeled and unlabeled pools ----
initial_ratio = 0.1
train_indices = np.arange(len(X_train))
y_train_np = y_train.numpy()

initial_indices, unlabeled_indices = train_test_split(
    train_indices, train_size=initial_ratio, stratify=y_train_np)

# ---- 3. Track Global Index (Optional but useful) ----
initial_indices_global = train_indices[initial_indices]
unlabeled_indices_global = train_indices[unlabeled_indices]

# ---- 4. Create pools ----
X_labeled = X_train[initial_indices]
y_labeled = y_train[initial_indices]

X_unlabeled_pool = X_train[unlabeled_indices]
y_unlabeled_pool = y_train[unlabeled_indices]

print(f"Initial labeled pool: {len(X_labeled)} samples")
print(f"Initial unlabeled pool: {len(X_unlabeled_pool)} samples\n")

# ---- 5. Normalize based on labeled training data ----
scalers = {}
for i in range(X_labeled.shape[2]):
    scalers[i] = StandardScaler()
    X_labeled[:, :, i] = torch.FloatTensor(scalers[i].fit_transform(X_labeled[:, :, i]))
    X_unlabeled_pool[:, :, i] = torch.FloatTensor(scalers[i].transform(X_unlabeled_pool[:, :, i]))
    X_test[:, :, i] = torch.FloatTensor(scalers[i].transform(X_test[:, :, i]))

# ---- 6. Create DataLoaders ----
batch_size = 64 # [1, 2, 4, 8, 16, 32, 64, 128, 256]
train_loader = DataLoader(TensorDataset(X_labeled, y_labeled), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=batch_size, shuffle=False)

# ---- 7. Label Distribution Printout ---
def print_label_distribution(labels, label_encoder, name=''):
    unique, counts = torch.unique(labels, return_counts=True)
    print(f"{name} label distribution:")
    for label, count in zip(unique.tolist(), counts.tolist()):
        print(f"  Label {label_encoder.inverse_transform([label])[0]}: {count} samples")
    return unique, counts

unique_train, counts_train = print_label_distribution(y_labeled, label_encoder, name='Labeled train set')
unique_test, counts_test = print_label_distribution(y_test, label_encoder, name='Test set')

# ---- 8. Display the selected indices
def display_selected_indices(selected_indices,  title="Selected Samples"):
    """
    Prints the names of selected samples given their indices.

    Args:
        selected_indices (array-like): Indices of selected samples.
        sample_names (list): List of sample names (indexed same as dataset).
        title (str): Optional title for the print output.
    """
    print(f"\n{title} ({len(selected_indices)} samples)")
    print(f'Selected indices: {selected_indices}')

display_selected_indices(initial_indices_global, title="Initial Labeled Indices")   

In [ ]:
def update_pools(selected_indices):
    
    """
    Update the labeled and unlabeled pools based on selected indices.
    Args:
        selected_indices (array-like): Indices of samples to move from unlabeled to labeled.
    Returns:
        DataLoader: Updated train DataLoader with new labeled data.
    """

    global X_labeled, y_labeled, X_unlabeled_pool, y_unlabeled_pool, train_loader

    # Move selected to labeled
    selected_X = X_unlabeled_pool[selected_indices]
    selected_y = y_unlabeled_pool[selected_indices]

    X_labeled = torch.cat([X_labeled, selected_X], dim=0)
    y_labeled = torch.cat([y_labeled, selected_y], dim=0)

    # Remove from unlabeled
    mask = torch.ones(len(X_unlabeled_pool), dtype=bool)
    mask[selected_indices] = False
    X_unlabeled_pool = X_unlabeled_pool[mask]
    y_unlabeled_pool = y_unlabeled_pool[mask]

    # Rebuild DataLoader
    train_loader = DataLoader(TensorDataset(X_labeled, y_labeled), batch_size=batch_size, shuffle=True)

    # Logging
    print(f"Updated labeled pool: {len(X_labeled)} samples")
    print(f"Remaining unlabeled pool: {len(X_unlabeled_pool)} samples")

    return train_loader # no need to return DataLoader, as we can access it globally

In [ ]:


def uncertainty_sampling(model, unlabeled_data, strategy, k, us_batch_size):
    """
    Perform uncertainty sampling to select the most uncertain samples from the unlabeled data.
    Args:
        model (nn.Module): The trained model to use for uncertainty estimation.
        unlabeled_data (Tensor): The unlabeled data from which to select samples.
        strategy (str): The uncertainty sampling strategy ('lc', 'sm', or 'entropy').
        k (int): The number of samples to select.
        us_batch_size (int): Batch size for processing the unlabeled data.
    Returns:
        Tuple: Indices of the selected samples and their corresponding probabilities.
        
    """

    model.eval()
    all_scores = []
    all_probs = []

    with torch.no_grad():
        for i in range(0, len(unlabeled_data), us_batch_size):
            batch = unlabeled_data[i:i + us_batch_size]
            logits = model(batch)
            probs = F.softmax(logits, dim=1)
            all_probs.append(probs)

            if strategy == 'lc':
                max_probs, _ = probs.max(dim=1)
                scores = 1 - max_probs

            elif strategy == 'sm':
                sorted_probs, _ = probs.sort(descending=True, dim=1)
                scores = -(sorted_probs[:, 0] - sorted_probs[:, 1])

            elif strategy == 'entropy':
                scores = -torch.sum(probs * torch.log(probs + 1e-12), dim=1)

            else:
                raise ValueError("Strategy must be 'lc', 'sm', or 'entropy'")

            all_scores.append(scores)

    all_scores = torch.cat(all_scores)
    all_probs = torch.cat(all_probs)

    # Get indices of top-k uncertain samples
    topk = torch.topk(all_scores, k=k)
    return topk.indices, all_probs[topk.indices]



In [ ]:
# Enhanced LSTM model with more hidden layers
class EnhancedToolLSTM(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, num_classes):
        super(EnhancedToolLSTM, self).__init__()
        # Three LSTM layers
        self.lstm1 = nn.LSTM(input_size, hidden_size1, batch_first=True)
        self.dropout1 = nn.Dropout(0.2)
        self.lstm2 = nn.LSTM(hidden_size1, hidden_size2, batch_first=True)
        self.dropout2 = nn.Dropout(0.2)
        self.lstm3 = nn.LSTM(hidden_size2, hidden_size3, batch_first=True)
        self.dropout3 = nn.Dropout(0.2)

        # Expanded fully connected layers
        self.fc1 = nn.Linear(hidden_size3, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 16)
        self.output = nn.Linear(16, num_classes)
        
        self.relu = nn.ReLU()
        self.leaky_relu = nn.LeakyReLU(0.1)

    def forward(self, x):
        out, _ = self.lstm1(x)
        out = self.dropout1(out)
        out, _ = self.lstm2(out)
        out = self.dropout2(out)
        out, _ = self.lstm3(out)
        out = self.dropout3(out)
        out = out[:, -1, :]  # Last time step

        out = self.leaky_relu(self.bn1(self.fc1(out)))
        out = self.leaky_relu(self.bn2(self.fc2(out)))
        out = self.leaky_relu(self.bn3(self.fc3(out)))
        out = self.relu(self.fc4(out))
        out = self.relu(self.fc5(out))
        out = self.output(out)
        return out

# Dynamic Focal Loss with adjustable class weights
class DynamicFocalLoss(nn.Module):
    def __init__(self, initial_weights=None, gamma=2.0, adjustment_rate=0.1, max_adjustment=0.5):
        super(DynamicFocalLoss, self).__init__()
        self.initial_weights = initial_weights
        self.weights = initial_weights.clone() if initial_weights is not None else None
        self.gamma = gamma
        self.adjustment_rate = adjustment_rate
        self.max_adjustment = max_adjustment
        self.class_performance = None

    def update_weights(self, class_performance):
        """Adjust weights based on class performance (accuracy per class)"""
        if self.weights is None:
            return
            
        # Calculate adjustment factor (boost weights for poorly performing classes)
        adjustment_factors = 1.0 - class_performance
        adjustment_factors = torch.clamp(adjustment_factors * self.adjustment_rate, 
                                      -self.max_adjustment, self.max_adjustment)
        
        # Apply adjustment
        new_weights = self.weights * (1 + adjustment_factors)
        new_weights = new_weights / new_weights.sum()  # Renormalize
        
        # Store for next epoch
        self.weights = new_weights
        self.class_performance = class_performance

    def forward(self, inputs, targets):
        logpt = nn.functional.log_softmax(inputs, dim=1)
        pt = torch.exp(logpt)
        logpt = logpt.gather(1, targets.view(-1, 1))
        pt = pt.gather(1, targets.view(-1, 1))

        if self.weights is not None:
            weights_t = self.weights.gather(0, targets)
            logpt = logpt * weights_t.view(-1, 1)

        loss = -1 * (1 - pt) ** self.gamma * logpt
        return loss.mean()
    

def train_test_lstm():
    """Train and test the LSTM model with dynamic focal loss and uncertainty sampling."""
      
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = EnhancedToolLSTM(input_size=X.shape[2], 
                            hidden_size1=128, 
                            hidden_size2=64, 
                            hidden_size3=32, 
                            num_classes=num_of_classes).to(device)

    # Initialize class weights (inverse of class frequencies)
    initial_weights = 1. / counts_train.float()
    initial_weights = initial_weights / initial_weights.sum()
    initial_weights = initial_weights.to(device)

    # Dynamic loss function
    criterion = DynamicFocalLoss(initial_weights=initial_weights, 
                            gamma=2.0, 
                            adjustment_rate=0.1,
                            max_adjustment=0.3)
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)

    # Initialize containers for tracking metrics
    train_losses, test_losses = [], []
    train_accuracies, test_accuracies = [], []
    class_performance_history = []  # To track per-class accuracy

    # Training loop with test evaluation each epoch
    num_epochs = 50
    for epoch in range(num_epochs):
        model.train()
        correct, total = 0, 0
        running_loss = 0.0
        class_correct = torch.zeros_like(initial_weights)
        class_total = torch.zeros_like(initial_weights)
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Track per-class accuracy
            for l in torch.unique(labels):
                mask = labels == l
                class_correct[l] += (predicted[mask] == labels[mask]).sum().item()
                class_total[l] += mask.sum().item()

        train_losses.append(running_loss / len(train_loader))
        train_accuracies.append(correct / total)
        
        # Calculate per-class accuracy
        class_performance = class_correct / class_total.clamp(min=1)  # Avoid division by zero
        class_performance_history.append(class_performance.cpu().numpy())
        
        # Update class weights based on performance
        criterion.update_weights(class_performance)

        # Evaluate on test set
        model.eval()
        test_loss = 0.0
        test_correct, test_total = 0, 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()
        test_losses.append(test_loss / len(test_loader))
        test_accuracies.append(test_correct / test_total)
        
        # Update learning rate
        scheduler.step(test_losses[-1])

        # Print epoch statistics
        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"  Train Loss: {train_losses[-1]:.4f}, Test Loss: {test_losses[-1]:.4f}")
        print(f"  Train Acc: {train_accuracies[-1]:.4f}, Test Acc: {test_accuracies[-1]:.4f}")
        print(f"  Class Performance: {dict(zip([label_encoder.inverse_transform([i])[0] for i in range(len(class_performance))], [f'{acc:.4f}' for acc in class_performance]))}")
        print(f"  Current Weights: {dict(zip([label_encoder.inverse_transform([i])[0] for i in range(len(criterion.weights))], [f'{w:.4f}' for w in criterion.weights]))}")
        print(f"  Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")

    # Final evaluation
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            y_true.extend(labels.numpy())
            y_pred.extend(predicted.cpu().numpy())

    # Metrics
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print(f"\nFinal Accuracy: {accuracy:.4f}")
    print(f"Final F1 Score: {f1:.4f}")

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    print("\nConfusion Matrix:")
    print(cm)

    TT = cm[0, 0]
    TF = cm[0, 1]
    FT = cm[1, 0]
    FF = cm[1, 1]
    print(f"TT (True tightening): {TT}")
    print(f"TF (Tightening predicted as untightening): {TF}")
    print(f"FT (Untightening predicted as tightening): {FT}")
    print(f"FF (True untightening): {FF}")

    # Classification Report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=['tightening', 'untightening']))

    # Plotting
    plt.figure(figsize=(18, 12))

    # Confusion Matrix
    plt.subplot(2, 2, 1)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['tightening', 'untightening'],
                yticklabels=['tightening', 'untightening'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')

    # Loss
    plt.subplot(2, 2, 2)
    plt.plot(train_losses, label='Training Loss')
    plt.plot(test_losses, label='Test Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss over Epochs')
    plt.legend()

    # Accuracy
    plt.subplot(2, 2, 3)
    plt.plot(train_accuracies, label='Training Accuracy')
    plt.plot(test_accuracies, label='Test Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Accuracy over Epochs')
    plt.legend()

    # Class Performance
    plt.subplot(2, 2, 4)
    class_perf_array = np.array(class_performance_history)
    for i in range(class_perf_array.shape[1]):
        plt.plot(class_perf_array[:, i], label=f'{label_encoder.inverse_transform([i])[0]} accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Class Accuracy')
    plt.title('Class-wise Accuracy over Epochs')
    plt.legend()

    plt.tight_layout()
    plt.show()

    return model, train_accuracies[-1], test_accuracies[-1], train_losses[-1], test_losses[-1]


In [ ]:
#--------------------------------- [START] Main Active Learning Loop -----------------------------------------------------------

# Active Learning Parameters
num_queries = 5            # How many active learning rounds
samples_per_query = 20     # How many samples to add per round
uncertainity_strategy = 'entropy'  # 'lc', 'sm', or 'entropy'
uncertainity_strategy_batch_size = 8  # Batch size for uncertainty sampling

strategy_map = {
    'lc': 'Least Confidence',
    'sm': 'Small Margin',
    'entropy': 'Entropy'
}

import pandas as pd

results_summary = pd.DataFrame(columns=[
    "Round", "AL_Method", "Training_Batch_Size","Uncertainity_Strategy_Batch_Size", "Num_Samples","Selected_Indices", "Train_Acc", "Test_Acc", "Train_Loss", "Test_Loss"
])


for num in range(num_queries):
    print(f"\n--- Active Learning Round {num + 1} ---")


    # Train model and collect metrics
    model, train_acc, test_acc, train_loss, test_loss = train_test_lstm()

    # Build row as a dict
    row_dict = {
        "Round": num + 1,
        "AL_Method": strategy_map.get(uncertainity_strategy, 'Unknown'),
        "Training_Batch_Size": batch_size,
        "Uncertainity_Strategy_Batch_Size": uncertainity_strategy_batch_size,
        "Num_Samples": len(y_labeled),
        "Selected_Indices": str(initial_indices_global.tolist()) if num == 0 else str(selected_indices.tolist()),
        "Train_Acc": round(train_acc * 100, 2),
        "Test_Acc": round(test_acc * 100, 2),
        "Train_Loss": round(train_loss, 4),
        "Test_Loss": round(test_loss, 4)
    }

    # Append row using loc
    results_summary.loc[len(results_summary)] = row_dict

    # Select most uncertain samples from unlabeled pool
    selected_indices, selected_probs = uncertainty_sampling(
    model, X_unlabeled_pool, strategy= uncertainity_strategy, k=samples_per_query, us_batch_size=uncertainity_strategy_batch_size
)
    # Display selected indices
    display_selected_indices(selected_indices, title=f"Selected Labeled Indices in round {num + 1}")

    # Update pools and train_loader
    train_loader = update_pools(selected_indices)

    print("Training samples count for weights initialization:", counts_train)
    
    # Optional: Check new label distribution
    unique_train, counts_train = print_label_distribution(y_labeled, label_encoder, name='Updated Labeled Pool')


In [ ]:
print("\n--- Summary of Active Learning Rounds ---")
print(results_summary)

# Save to CSV
csv_path = "active_learning_results.csv"
if not os.path.exists(csv_path):
    results_summary.to_csv(csv_path, index=False)
else:
    results_summary.to_csv(csv_path, mode='a', header=False, index=False)
print(f"\nResults saved to {csv_path}")
